In [2]:
import polars as pl
import numpy as np

In [38]:
df = pl.scan_parquet('./Datasets/crsp')
df.head().collect()

PERMNO,date,NAMEENDT,SHRCD,EXCHCD,NCUSIP,TICKER,COMNAM,SHRCLS,PRIMEXCH,TRDSTAT,SECSTAT,PERMCO,CUSIP,DCLRDT,DLAMT,DLPDT,DLSTCD,PAYDT,RCRDDT,SHRFLG,DISTCD,DIVAMT,FACPR,FACSHR,ACPERM,ACCOMP,SHRENDDT,NWPERM,DLRETX,DLPRC,DLRET,BIDLO,ASKHI,PRC,VOL,RET,ASK,SHROUT,CFACPR,CFACSHR,OPENPRC,NUMTRD,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
i64,str,str,i64,i64,str,str,str,str,str,str,str,i64,str,str,f64,str,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64
10001,"""2004-01-02""",null,11,3,"""29274A10""","""EWST""","""ENERGY WEST INC""",null,"""Q""","""A""","""R""",7953,"""36720410""",null,null,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,6.0,6.87,6.55,9470.0,"""0.100840""",6.59,2596.0,1.5,1.5,6.0,25.0,"""0.100840""",-0.000784,-0.000859,0.00607,0.006048,-0.003094
10001,"""2004-01-05""",null,11,3,"""29274A10""","""EWST""","""ENERGY WEST INC""",null,"""Q""","""A""","""R""",7953,"""36720410""",null,null,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,6.44,6.75,6.67,7516.0,"""0.018321""",6.95,2596.0,1.5,1.5,6.44,22.0,"""0.018321""",0.012269,0.012264,0.012936,0.012927,0.012395
10001,"""2004-01-06""",null,11,3,"""29274A10""","""EWST""","""ENERGY WEST INC""",null,"""Q""","""A""","""R""",7953,"""36720410""",null,null,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,6.65,7.0,6.76,5743.0,"""0.013493""",6.74,2596.0,1.5,1.5,7.0,23.0,"""0.013493""",0.001951,0.001917,0.004868,0.004855,0.001292
10001,"""2004-01-07""",null,11,3,"""29274A10""","""EWST""","""ENERGY WEST INC""",null,"""Q""","""A""","""R""",7953,"""36720410""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.75,6.8,6.75,6825.0,"""-0.001479""",6.8,2596.0,1.5,1.5,6.8,19.0,"""-0.001479""",0.002716,0.00252,0.005653,0.005574,0.002367
10001,"""2004-01-08""",null,11,3,"""29274A10""","""EWST""","""ENERGY WEST INC""",null,"""Q""","""A""","""R""",7953,"""36720410""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.65,6.85,6.74,33624.0,"""-0.001482""",6.83,2596.0,1.5,1.5,6.85,44.0,"""-0.001482""",0.004865,0.004831,0.008348,0.008317,0.004963


In [ ]:
# efficient daily log returns for Apple
aapl_permno = df.filter(pl.col('TICKER') == 'AAPL') \
    .select('PERMNO') \
    .unique() \
    .collect() \
    .item()

log_returns = df.filter(pl.col('PERMNO') == aapl_permno) \
    .select(['PERMNO', 'PRC']) \
    .with_columns(
        (pl.col('PRC')/pl.col('PRC').shift(1)) \
            .log()
            .alias('LOG_RET')
        ) \
    .collect()

PERMNO,PRC,LOG_RET
i64,f64,f64
14593,21.28,null
14593,22.17,0.040972
14593,22.09,-0.003615
14593,22.59,0.022382
14593,23.36,0.033518
…,…,…
14593,258.20001,0.011413
14593,259.01999,0.003171
14593,255.59,-0.013331


In [17]:
df_cstat = pl.scan_parquet('./Datasets/compustat/fundamentals_quarterly_all')
cstate_len = df_cstat.select([pl.len()]).collect().item()
